In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# Eğitim ve test kümelerinin oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))

lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train.reshape(X_train.shape[0], -1), y_train)
X_test_lda = lda.transform(X_test.reshape(X_test.shape[0], -1))

# CNN Modeli Tanımlama
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(64, kernel_size=2, activation='relu'),
        MaxPooling1D(pool_size=1),  # pool_size zaman boyutuna uyumlu olacak şekilde ayarlandı
        Flatten(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performans Hesaplama
results = []

for reduction, X_tr, X_te, y_tr, y_te in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_train, X_train_pca, X_train_lda],
    [X_test, X_test_pca, X_test_lda],
    [y_train, y_train, y_train],
    [y_test, y_test, y_test]
):
    if reduction == 'No PCA/LDA':
        X_tr = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], X_tr.shape[2])  # 3D giriş
        X_te = X_te.reshape(X_te.shape[0], X_te.shape[1], X_te.shape[2])
    else:
        # PCA ve LDA için uygun giriş boyutları ayarlanıyor
        X_tr = np.tile(X_tr, (1, SEQ_LENGTH)).reshape(X_tr.shape[0], SEQ_LENGTH, -1)
        X_te = np.tile(X_te, (1, SEQ_LENGTH)).reshape(X_te.shape[0], SEQ_LENGTH, -1)

    cnn_model = build_cnn_model(X_tr.shape[1:])
    history = cnn_model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1, validation_data=(X_te, y_te))

    # Eğitim ve Test Kayıpları
    train_loss, train_accuracy = history.history['loss'][-1], history.history['accuracy'][-1] * 100
    test_loss, test_accuracy = history.history['val_loss'][-1], history.history['val_accuracy'][-1] * 100

    results.append({
        'Reduction': reduction,
        'Model': 'CNN',
        'Train Accuracy (%)': train_accuracy,
        'Test Accuracy (%)': test_accuracy,
        'Train Loss': train_loss,
        'Test Loss': test_loss
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# Performans Sonuçları Tablosu
print("Performans Sonuçları Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("cnn_performance_results.csv", index=False)
print("\nSonuçlar 'cnn_performance_results.csv' dosyasına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8440 - loss: 0.3724 - val_accuracy: 0.8684 - val_loss: 0.3063
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8775 - loss: 0.2973 - val_accuracy: 0.8737 - val_loss: 0.2949
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8780 - loss: 0.2851 - val_accuracy: 0.8757 - val_loss: 0.2913
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8782 - loss: 0.2803 - val_accuracy: 0.8724 - val_loss: 0.2896
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8803 - loss: 0.2793 - val_accuracy: 0.8732 - val_loss: 0.2926
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8817 - loss: 0.2755 - val_accuracy: 0.8762 - val_loss: 0.2887
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8842 - loss: 0.2723 - val_accuracy: 0.8740 - val_loss: 0.2904
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8855 - loss: 0.2675 - 